In [157]:
import os
import shutil

import json
import ruamel_yaml as yaml
import conda_pack

from subprocess import check_call, check_output
from conda.cli.python_api import Commands, run_command
from jinja2 import Environment, FileSystemLoader

# Create the JupyterHub Environment

In [158]:
with open('environment.yml') as f:
    conda_env = yaml.load(f, Loader=yaml.Loader)

In [159]:
check_call("conda env create --force -p {name}".format(**conda_env).split(' '))

0

# Add the `parcel.json` and script to `prefix/meta` directory

In [160]:
meta_dir = os.path.join(conda_env_name, "meta")
os.makedirs(meta_dir, exist_ok=True)

In [161]:
shutil.copy('jupyterhub_env.sh', meta_dir)

'jupyterhub/meta/jupyterhub_env.sh'

In [162]:
components = json.loads(run_command(Commands.LIST, "--json", "-p", conda_env['name'])[0])

In [163]:
main_version = [x['version'] for x in components if x['name'] == conda_env['name']][0]

In [164]:
jinja_env = Environment(loader=FileSystemLoader("templates"))
parcel_template = jinja_env.get_template("parcel.yaml")
parcel_rendered = parcel_template.render(conda_env=conda_env,
                                         version=main_version,
                                         components=components)
parcel_dict = yaml.load(parcel_rendered, Loader=yaml.Loader)

parcel_json = os.path.join(meta_dir, "parcel.json")
with open(parcel_json, 'w') as f:
    json.dump(parcel_dict, f)
    
print(check_output(['java', '-jar', 'cm_ext/validator/target/validator.jar', '-p', parcel_json]).decode('utf-8'))

Validating: jupyterhub/meta/parcel.json
Validation succeeded.



# Use Conda-Pack to create .parcel file

In [165]:
shutil.rmtree('parcels')
os.makedirs('parcels', exist_ok=True)
parcel_fqn = '{name}-{version}'.format(**parcel_dict)
parcel_file = conda_pack.pack(prefix=conda_env_name,
                         output='%s-distro.parcel' % parcel_fqn,
                         arcroot=parcel_fqn,
                         dest_prefix="/opt/cloudera/parcels/%s" % parcel_fqn,
                         format="tar.gz",
                         force=True)

# Link generated file to different distros

In [166]:
distros=["el6", "el7"]
for distro in distros:
    dest_name = "parcels/{}-{}.parcel".format(parcel_fqn, distro)
    os.symlink("../%s-distro.parcel" % parcel_fqn, dest_name)
    check_call(['java', '-jar', 'cm_ext/validator/target/validator.jar', '-f', dest_name])

# Generate parcel manifest file

In [167]:
print(check_output(['python', 'cm_ext/make_manifest/make_manifest.py', 'parcels']).decode('utf-8'))

Scanning directory: parcels
Found parcel JUPYTERHUB-0.9.6-0.0.3-el6.parcel
Found parcel JUPYTERHUB-0.9.6-0.0.3-el7.parcel

